In [1]:
# IMPORTS
import pandas as pd

from enron.src.common.const import *

from manufacturing_company.src.common.const import *
from manufacturing_company.src.features.employee_activity import *
from manufacturing_company.src.features.neighborhood_variability import *


In [2]:
# CREATE FEATURES
for month in MONTHS:
    df = pd.read_csv(ENRON_FILE_MINIMUM_ACTIVITY.format(month), sep=';', parse_dates=[EVENT_DATE])
    
    overtime = calculate_overtime(df, from_hour=17, to_hour=5)
    work_at_weekend = calculate_work_at_weekend(df)
    
    features = pd.merge(overtime, work_at_weekend, on=SENDER)
    
    network_measures = pd.read_csv(ENRON_FILE_NETWORK_MEASURES.format(month), sep=';')
    
    features = pd.merge(network_measures, features, left_on=ID, right_on=SENDER)
    
    nv_sender, nv_recipient, nv_sender_recipient = calculate_neighborhood_variability(df, features[ID])
    
    nv_sender = pd.DataFrame.from_dict(nv_sender, orient='index', columns=[NEIGHBORHOOD_VARIABILITY_SENDER])
    nv_sender.index.name = ID
    
    nv_recipient = pd.DataFrame.from_dict(nv_recipient, orient='index', columns=[NEIGHBORHOOD_VARIABILITY_RECIPIENT])
    nv_recipient.index.name = ID
    
    nv_sender_recipient = pd.DataFrame.from_dict(nv_sender_recipient, orient='index', columns=[NEIGHBORHOOD_VARIABILITY_ALL])
    nv_sender_recipient.index.name = ID
    
    features = pd.merge(features, nv_sender, on=ID)
    features = pd.merge(features, nv_recipient, on=ID)
    features = pd.merge(features, nv_sender_recipient, on=ID)
    
    features.to_csv(ENRON_FILE_FEATURES.format(month), sep=';', index=False)
